This can be executed in https://colab.research.google.com "Python 3 / GPU" runtime.

In [ ]:
!git clone https://github.com/clovaai/deep-text-recognition-benchmark
%cd deep-text-recognition-benchmark

Cloning into 'deep-text-recognition-benchmark'...
remote: Enumerating objects: 499, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 499 (delta 0), reused 1 (delta 0), pack-reused 495
Receiving objects: 100% (499/499), 3.07 MiB | 6.88 MiB/s, done.
Resolving deltas: 100% (301/301), done.
/content/deep-text-recognition-benchmark


Next, download large model files from Google Drive, using hack: https://stackoverflow.com/questions/20665881/direct-download-from-google-drive-using-google-drive-api/32742700#32742700

In [ ]:
models = {
    'None-ResNet-None-CTC.pth': 'https://drive.google.com/open?id=1FocnxQzFBIjDT2F9BkNUiLdo1cC3eaO0',
    'None-VGG-BiLSTM-CTC.pth': 'https://drive.google.com/open?id=1GGC2IRYEMQviZhqQpbtpeTgHO_IXWetG',
    'None-VGG-None-CTC.pth': 'https://drive.google.com/open?id=1FS3aZevvLiGF1PFBm5SkwvVcgI6hJWL9',
    'TPS-ResNet-BiLSTM-Attn-case-sensitive.pth': 'https://drive.google.com/open?id=1ajONZOgiG9pEYsQ-eBmgkVbMDuHgPCaY',
    'TPS-ResNet-BiLSTM-Attn.pth': 'https://drive.google.com/open?id=1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9',
    'TPS-ResNet-BiLSTM-CTC.pth': 'https://drive.google.com/open?id=1FocnxQzFBIjDT2F9BkNUiLdo1cC3eaO0',
}

for k, v in models.items():
  doc_id = v[v.find('=')+1:]
  !curl -c /tmp/cookies "https://drive.google.com/uc?export=download&id=$doc_id" > /tmp/intermezzo.html
  !curl -L -b /tmp/cookies "https://drive.google.com$(cat /tmp/intermezzo.html | grep -Po 'uc-download-link" [^>]* href="\K[^"]*' | sed 's/\&amp;/\&/g')" > $k

!ls -al *.pth

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2288    0  2288    0     0   8633      0 --:--:-- --:--:-- --:--:--  8666
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   558  100   558    0     0    783      0 --:--:-- --:--:-- --:--:--   783
100  580k    0  580k    0     0   641k      0 --:--:-- --:--:-- --:--:--  641k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  % Total    % Received % Xferd  Average Speed   Tim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install torch torchvision


In [ ]:
# downloading the dataset
!wget -O dataset.zip "https://www.dropbox.com/sh/i39abvnefllx2si/AAAbAYRvxzRp3cIE5HzqUw3ra?dl=1"
!unzip dataset.zip -d ./dataset


--2023-11-26 15:53:48--  https://www.dropbox.com/sh/i39abvnefllx2si/AAAbAYRvxzRp3cIE5HzqUw3ra?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /sh/dl/i39abvnefllx2si/AAAbAYRvxzRp3cIE5HzqUw3ra [following]
--2023-11-26 15:53:48--  https://www.dropbox.com/sh/dl/i39abvnefllx2si/AAAbAYRvxzRp3cIE5HzqUw3ra
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4396385310282b4ef71abf4ec3.dl.dropboxusercontent.com/zip_download_get/Brq4gQvNfJs_ZGR-_GynUnpoSwpdApKLkmwHdceTQs1fYt9p5CzYsEnXXprFL0tx8SyWWacMdE_LyMHvSdp7VqddYmcTz6LarSgx73LoxIW6Fg?dl=1# [following]
--2023-11-26 15:53:48--  https://uc4396385310282b4ef71abf4ec3.dl.dropboxusercontent.com/zip_download_get/Brq4gQvNfJs_ZGR-_GynUnpoSwpdApKLkmwHdceTQs1fYt9p5CzYsEnXXprFL0tx8SyWWacMdE_LyMHvSdp7VqddY

In [ ]:
# requirements
!pip3 install lmdb pillow torchvision nltk natsort

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 4.2 MB/s eta 0:00:00


In [ ]:
# download the pre-trained model
!wget -O TPS-ResNet-BiLSTM-Attn.pth "https://drive.google.com/file/d/1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9/view?usp=drive_link"

--2023-11-26 16:26:06--  https://drive.google.com/file/d/1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9/view?usp=drive_link
Resolving drive.google.com (drive.google.com)... 142.251.2.139, 142.251.2.102, 142.251.2.100, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘TPS-ResNet-BiLSTM-Attn.pth’

TPS-ResNet-BiLSTM-A     [ <=>                ]  81.93K  --.-KB/s    in 0.005s  

2023-11-26 16:26:06 (15.5 MB/s) - ‘TPS-ResNet-BiLSTM-Attn.pth’ saved [83900]



In [ ]:
!gdown --id 1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9 -O TPS-ResNet-BiLSTM-Attn.pth # that's the id of the model from the google drive link from above


/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9
To: /content/deep-text-recognition-benchmark/TPS-ResNet-BiLSTM-Attn.pth
100% 199M/199M [00:03<00:00, 60.2MB/s]


In [ ]:
!CUDA_VISIBLE_DEVICES=0 python3 demo.py \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--image_folder ./dataset/demo_image/ \
--saved_model TPS-ResNet-BiLSTM-Attn.pth

model input parameters 32 100 20 1 512 256 38 25 TPS ResNet BiLSTM Attn
loading pretrained model from TPS-ResNet-BiLSTM-Attn.pth


In [ ]:
!python3 demo.py \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--image_folder ./demo_image/ \
--saved_model TPS-ResNet-BiLSTM-Attn.pth

I got here before main function
model input parameters 32 100 20 1 512 256 38 25 TPS ResNet BiLSTM Attn
loading pretrained model from TPS-ResNet-BiLSTM-Attn.pth
after the model.eval
hello, i am in no_grad
Number of batches in loader: 1
--------------------------------------------------------------------------------
image_path               	predicted_labels         	confidence score
--------------------------------------------------------------------------------
./demo_image/demo_1.png  	available                	0.9999
./demo_image/demo_2.jpg  	shakeshack               	0.9530
./demo_image/demo_3.png  	london                   	0.9840
./demo_image/demo_4.png  	greenstead               	0.9985
./demo_image/demo_5.png  	toast                    	0.9961
./demo_image/demo_6.png  	merry                    	0.9975
./demo_image/demo_7.png  	underground              	1.0000
./demo_image/demo_8.jpg  	ronaldo                  	0.8387
./demo_image/demo_9.jpg  	bally                    	0.7493
./

**observations**:

The lowest confidence score are for a curved half-circle words.


In [ ]:
%pwd

'/content/deep-text-recognition-benchmark'

In [ ]:
%ls

create_lmdb_dataset.py  LICENSE.md                test.py
dataset/                model.py*                 TPS-ResNet-BiLSTM-Attn-case-sensitive.pth
dataset.py              modules/                  TPS-ResNet-BiLSTM-Attn.pth
dataset.zip             None-ResNet-None-CTC.pth  TPS-ResNet-BiLSTM-CTC.pth
demo_image/             None-VGG-BiLSTM-CTC.pth   train.py
demo.ipynb              None-VGG-None-CTC.pth     utils.py
demo.py*                __pycache__/
figures/                README.md


In [ ]:
%cd dataset

/content/deep-text-recognition-benchmark/dataset


In [ ]:
%cd /content/deep-text-recognition-benchmark

/content/deep-text-recognition-benchmark


**Please remember to correct the paths or similar ... to not spend time on debugging for nothing**

In [ ]:
# also change the num_workers to 2, because that's what the GPU has

AttributeError: ignored

## Augmentations:

In [ ]:
# i think it's easier to make a new python file of the same style of dataset.py with a new class Dataset like rawDataset
# to make the augmentations
# then we call it in the bash like above

In [ ]:
# after applying the changes

In [ ]:
%cd ..
# you should be in /content/deep-text-recognition-benchmark
# to run the demo.py thing

/content/deep-text-recognition-benchmark


In [ ]:
!python3 demo.py \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--image_folder ./demo_image/ \
--saved_model TPS-ResNet-BiLSTM-Attn.pth

I got here before main function
model input parameters 32 100 20 1 512 256 38 25 TPS ResNet BiLSTM Attn
loading pretrained model from TPS-ResNet-BiLSTM-Attn.pth
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1284: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
i made the augdataset
after the model.eval
hello, i am in no_grad
Number of batches in loader: 1
--------------------------------------------------------------------------------
image_path               	predicted_labels         	confidence score
--------------------------------------------------------------------------------
./demo_image/demo_1.png  	sidelisva                	0.5261
./demo_image/demo_2.jpg  	yoarzzair                	0.0001
./demo_image/demo_3.png  	nobadi                   	0.0766
./demo_image/demo_4.png  	greenstead               	0.9985
./demo_image/demo_5.png  	tzaot                    	0.3119
./demo_image/demo_6.png